# TEXT TO SIGN LANGUAGE

In [ ]:
#pip install gtts

In [ ]:
#pip install pygame

In [4]:
import os
import cv2
import pygame
import json
import time
import numpy as np
from gtts import gTTS
import nltk
from nltk.tokenize import word_tokenize

# Load the mapping JSON
json_path = r"C:\Users\Athira\OneDrive\Documents\Desktop\SIGN_LANGUAGE\mapping.json"
with open(json_path, 'r') as json_file:
    mapping = json.load(json_file)

# Extract mappings
d1 = {key.lower(): value for key, value in mapping['alphabets'].items()}  
d2 = {key.lower(): value for key, value in mapping['words'].items()}  

# Create output directory if it doesn't exist
output_dir = r"C:\Users\Athira\OneDrive\Documents\Desktop\SIGN_LANGUAGE\OUTPUT"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

video_files = []  # Store video file paths

# Initialize pygame mixer
pygame.mixer.init()

def play_video_with_audio(video_path, text):
    
    if not os.path.exists(video_path):
        print(f"Error: Cannot find video file {video_path}")
        return None
    
    # Generate speech audio for the text
    audio_path = os.path.join(output_dir, f"{text}.mp3")
    tts = gTTS(text=text, lang='en')
    tts.save(audio_path)

    # Load and play audio
    pygame.mixer.music.load(audio_path)
    pygame.mixer.music.play()

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return None  

    # Get video properties (but will resize all to 640x640)
    width = 640  # Fixed width
    height = 640  # Fixed height
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Store frames for concatenation
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (width, height))  # Resize all frames to 640x640
        frames.append(frame)  # Collect frames

        cv2.imshow('Video', frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    pygame.mixer.music.stop()
    
    return frames, width, height, fps  # Return frames for later concatenation

# Process user input
sentence = input("ENTER A SENTENCE: ").strip().lower()
words = word_tokenize(sentence)

all_frames = []
final_width, final_height, final_fps = 640, 640, 20  # Set final size to 640x640 and default FPS to 20

for word in words:
    if word in d2:
        print(f"Processing word: {word}")
        frames, width, height, fps = play_video_with_audio(d2[word], word)
    else:
        print(f"Word '{word}' not found. Processing letters.")
        frames, width, height, fps = [], None, None, None
        for letter in word:
            if letter in d1:
                print(f"Processing letter: {letter}")
                letter_frames, width, height, fps = play_video_with_audio(d1[letter], letter)
                frames.extend(letter_frames)
            else:
                print(f"No video found for letter '{letter}'.")

    if frames:
        all_frames.extend(frames)

# Save the final concatenated video
if all_frames:
    output_video_path = os.path.join(output_dir, f"{sentence.replace(' ', '_')}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, final_fps, (final_width, final_height))

    # Write all frames to the final video
    for frame in all_frames:
        out.write(frame)
    out.release()

    print(f"Final video saved: {output_video_path}")
else:
    print("No valid videos were processed.")

print("Processing complete!")


ENTER A SENTENCE:  good morning


Processing word: good
Error: Cannot find video file C:\Users\Dell\Desktop\KAREEM\SIGN_LANG\MEDIAPIPE\PHASE_2\sign_mp4\good.mp4


TypeError: cannot unpack non-iterable NoneType object

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Athira\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True